In [1]:
import sys 
sys.path.append('..')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Сравнение с климатикой

In [3]:
min_temp = pd.read_excel('../input/MinClimate.xlsx', sheet_name='Temp')
min_prec = pd.read_excel('../input/MinClimate.xlsx', sheet_name='Prec')

In [4]:
filtered_prec = min_prec[min_temp>=10]
filtered_prec['Month'] = min_prec['Month']
filtered_prec['Day'] = min_prec['Day']
filtered_temp = min_temp[min_temp>=10]
filtered_temp['Month'] = min_temp['Month']
filtered_temp['Day'] = min_temp['Day']

In [5]:
monthly_temp = pd.DataFrame({'Year':list(min_temp.columns[2:])})
monthly_prec = pd.DataFrame({'Year':list(min_prec.columns[2:])})
monthly_htc = pd.DataFrame({'Year':list(min_prec.columns[2:])})

months_names = ['', 'January', 'February', 'March',
                'April', 'May', 'June', 'July',
                'August', 'September', 'October',
                'November', 'December']

for i in range(1,13):
    monthly_htc[months_names[i]] = list(filtered_prec[filtered_prec['Month']==i].sum(skipna=True)[2:] * 10 / filtered_temp[filtered_temp['Month']==i].sum(skipna=True)[2:])
    monthly_temp[months_names[i]] = list(min_temp[min_temp['Month']==i].mean(skipna=True)[2:])
    monthly_prec[months_names[i]] = list(min_prec[min_prec['Month']==i].sum(skipna=True)[2:])

In [6]:
monthly_vpd = pd.read_csv('../input/minusinsk_VPD/wr87011.txt', delimiter=';')

In [7]:
a_classified = pd.read_excel('../input/quotient_deviation_df_A_CLASSIFIED.xlsx')
b_classified = pd.read_excel('../input/quotient_deviation_df_B_CLASSIFIED_FIXED.xlsx')

In [8]:
def get_moving_avg(df, window=7, nanmean=False):
    """
    Возвращает скользящее средние для температуры
    window : окно
    nanmean : используем nanmean для сглаживания? (тогда потеряются данные по краям)
    """
    month = df['Month']
    day = df['Day']
    if nanmean:
        result = df.rolling(window=window, center=True).apply(np.nanmean)
    else:
        result = df.rolling(window=window, center=True, min_periods=1).mean()
    result['Month'] = month
    result['Day'] = day
    return result

def get_moving_sum(df, window=7):
    """
    Возвращает скользящую сумму для осадков
    window : окно
    """
    month = df['Month']
    day = df['Day']
    result = df.rolling(window=window, center=True, min_periods=1).sum()
    result['Month'] = month
    result['Day'] = day
    return result

def list_to_cumulative(lst):
    return [sum(lst[:i+1]) for i in range(len(lst))]

In [9]:
import matplotlib.dates as mdates
from datetime import date

In [10]:
first_day = 91 +30 #, +44, +61
last_day = -61 -31

In [12]:
date_df = min_temp[['Month', 'Day']]
date_df['Year'] = [2000 for _ in range(len(date_df))]
x = pd.to_datetime(date_df)[first_day:last_day]

for i in range(4):
    mean_temp = get_moving_avg(min_temp)[[year for year in a_classified[a_classified['Class 4']==i]['Year'] if year in min_temp.columns]]
    y = mean_temp.mean(axis=1)[first_day:last_day]
    dy = mean_temp.std(axis=1)[first_day:last_day]

    fig, ax = plt.subplots(nrows=3, ncols=1, dpi=300, figsize=(6, 8))
    plt.subplots_adjust(bottom=0.03, top=0.95)

    ax[0].fill_between(x, y-dy, y+dy, color='red', alpha=0.2)
    ax[0].plot(x, y, c='red')

    mean_prec = get_moving_avg(min_prec)[[year for year in a_classified[a_classified['Class 4']==i]['Year'] if year in min_prec.columns]]
    y = mean_prec.mean(axis=1)[first_day:last_day]
    dy = mean_prec.std(axis=1)[first_day:last_day]
    ax2 = ax[0].twinx()
    #ax2.fill_between(x, [0 for _ in y], list_to_cumulative(y), color='blue', alpha=0.1)
    ax2.plot(x, list_to_cumulative(y), color='blue')
    ax2.set_ylim([0, 350])
    
    ax[0].set_ylabel('Temperature (°C)')
    ax2.set_ylabel('Precipitation (mm)')
    ax[0].set_zorder(1)  # default zorder is 0 for ax1 and ax2
    ax[0].patch.set_visible(False)  # prevents ax1 from hiding ax2

    locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
    formatter = mdates.ConciseDateFormatter(locator)
    ax[0].xaxis.set_major_locator(locator)
    ax[0].xaxis.set_major_formatter(formatter)
    ax[0].set_xlim([date(2000, 5, 1), date(2000, 9, 30)])
    ax[0].set_ylim([0, 25])
    selected = a_classified[a_classified['Class 4']==i]
    temp_to_plot = monthly_temp.loc[monthly_temp['Year'].isin(list(selected['Year']))]
    

    for _, row in selected.iterrows():
        ax[1].plot(list(row[1:16]), color='lightgray')
        ax[2].plot(list(row[16: 31]), color='lightgray')
    
    ax[1].plot(selected.mean()[1:16], label='Diameter deviation', color='black')
    ax[2].plot([f'C{j}' for j in range(1,16)], selected.mean()[16:31], label='CWT deviation', color='black')
    ax[1].axhline(y=1, color='dimgray', linestyle='--')
    ax[2].axhline(y=1, color='dimgray', linestyle='--')
    #ax[1].legend(frameon=False)
    ax[1].legend(frameon=False)
    ax[2].legend(frameon=False)
    ax[1].set_ylim([0.65, 1.35])
    ax[2].set_ylim([0.65, 1.35])
    ax[0].set_title(f'{i+1} Class')
    plt.savefig(f'../output/{i+1} Class (d&cwt)', dpi=300)
    plt.close(fig)
    #plt.show()

C:\Users\Mikhail\AppData\Local\Temp/ipykernel_5044/2870294080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df['Year'] = [2000 for _ in range(len(date_df))]


# ANOVA

## ANOVA for Temperature

In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [16]:
temp_to_anova = []
for i in range(0,4):
    mean_temp = get_moving_avg(min_temp)[[year for year in a_classified[a_classified['Class 4']==i]['Year'] if year in min_temp.columns]]
    _temp_to_anova = mean_temp[7::7].T.reset_index().rename({'index':'Year'}, axis=1)
    _temp_to_anova['Class'] = [i+1] * len(_temp_to_anova)
    temp_to_anova += [_temp_to_anova]
temp_to_anova = pd.concat(temp_to_anova)

In [18]:
_ps = []
for j in range(12,46):
    lm = ols(f'Q({7*j}) ~ Class',data=temp_to_anova).fit()
    _p = sm.stats.anova_lm(lm)['PR(>F)']['Class']
    _ps += [_p]
    if _p <0.05:
        print(f"{j}: {_p:.2f}")

12: 0.02
16: 0.03
17: 0.02
18: 0.02
20: 0.00
23: 0.04
27: 0.04
28: 0.05
29: 0.01
30: 0.04


## ANOVA for Precipitation

In [19]:
prec_to_anova = []
for i in range(0,4):
    mean_prec = get_moving_sum(min_prec)[[year for year in a_classified[a_classified['Class 4']==i]['Year'] if year in min_prec.columns]]
    _prec_to_anova = mean_prec[7::7].T.reset_index().rename({'index':'Year'}, axis=1)
    _prec_to_anova['Class'] = [i+1] * len(_prec_to_anova)
    prec_to_anova += [_prec_to_anova]
prec_to_anova = pd.concat(prec_to_anova)

In [20]:
for j in range(12,46):
    lm = ols(f'Q({7*j}) ~ Class',data=prec_to_anova).fit()
    _p = sm.stats.anova_lm(lm)['PR(>F)']['Class']
    if _p <0.05:
        print(f"{j}: {_p:.2f}")

21: 0.01
40: 0.01


# Kruskal–Wallis test

In [21]:
from scipy.stats import mannwhitneyu

In [22]:
from scipy.stats import mstats

def get_class_values(df, cl, col):
    return list(df[df['Class']==cl][col])

print('Temp')
for column in temp_to_anova.columns:

    if column == 'Year' or column == 'Class':
        continue
    s, p = mstats.kruskalwallis(get_class_values(temp_to_anova, 1, column),
                                get_class_values(temp_to_anova, 2, column),
                                get_class_values(temp_to_anova, 3, column),
                                get_class_values(temp_to_anova, 4, column))
    if p < 0.05:
        print(column//7, s, p)

print('Prec')
for column in prec_to_anova.columns:

    if column == 'Year' or column == 'Class':
        continue
    s, p = mstats.kruskalwallis(get_class_values(prec_to_anova, 1, column),
                                get_class_values(prec_to_anova, 2, column),
                                get_class_values(prec_to_anova, 3, column),
                                get_class_values(prec_to_anova, 4, column))
    if p < 0.05:
        print(column//7, s, p)

Temp
9 8.861712011284014 0.031187139969827127
10 11.17474379650355 0.010817540652059943
12 7.904339870028564 0.048030641113517375
20 9.726197106405545 0.021042855954674623
21 9.028513499481512 0.02891417633118446
Prec
7 9.800523948779903 0.02034012639612841
10 10.300988175227852 0.01617350100074091
19 8.128461909957418 0.04343006731911003
29 10.251322663533552 0.01654627910602947
44 8.658625207132223 0.03419143862089558
52 8.05647417807848 0.04485886006398594
